In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
183,AFG,Asia,Afghanistan,2020-07-11,34366.0,458.0,994.0,37.0,882.802,11.765,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
308,ALB,Europe,Albania,2020-07-11,3278.0,90.0,85.0,2.0,1139.065,31.274,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
497,DZA,Africa,Algeria,2020-07-11,17808.0,0.0,988.0,0.0,406.102,0.000,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-10,France,170829.0,0.0,30009.0,0.0,0.0,0.0
0,2020-07-10,China,85062.0,0.0,4650.0,0.0,0.0,0.0
0,2020-07-10,Italy,242686.0,0.0,34965.0,0.0,0.0,0.0
0,2020-07-10,Spain,253492.0,0.0,28418.0,0.0,0.0,0.0
0,2020-07-10,United States,3168255.0,0.0,134221.0,0.0,0.0,0.0
0,2020-07-10,World,12456532.0,0.0,560718.0,0.0,0.0,0.0
0,2020-07-10,United Kingdom,288628.0,0.0,44771.0,0.0,0.0,0.0
0,2020-07-10,Germany,198684.0,0.0,9074.0,0.0,0.0,0.0
0,2020-07-10,Iran,253953.0,0.0,12441.0,0.0,0.0,0.0
0,2020-07-10,Turkey,210955.0,0.0,5321.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-11,France,170829.0,170752.0,30009.0,30004.0,77.0,5.0
0,2020-07-11,China,85062.0,85036.0,4650.0,4641.0,26.0,9.0
0,2020-07-11,Italy,242686.0,242639.0,34965.0,34938.0,47.0,27.0
0,2020-07-10,Spain,253492.0,253908.0,28418.0,28403.0,-416.0,15.0
0,2020-07-11,United States,3168255.0,3184633.0,134221.0,134097.0,-16378.0,124.0
0,2020-07-11,World,12456532.0,12476028.0,560718.0,559998.0,-19496.0,720.0
0,2020-07-11,United Kingdom,288628.0,288133.0,44771.0,44650.0,495.0,121.0
0,2020-07-11,Germany,198684.0,198556.0,9074.0,9060.0,128.0,14.0
0,2020-07-11,Iran,253953.0,252720.0,12441.0,12447.0,1233.0,-6.0
0,2020-07-11,Turkey,210955.0,210965.0,5321.0,5323.0,-10.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")